<a href="https://colab.research.google.com/github/zyz314/100-Days-Of-ML-Code/blob/master/project6_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TheoreticallyOptimalStrategy.py

# 导入必要的库
import pandas as pd  # 用于数据处理
import numpy as np  # 用于数值计算
import datetime as dt

import util
from util import get_data, plot_data

def author():
  return "yzheng438"

# 模块 1: 获取股票数据函数
def get_stock_data(symbol, start_date, end_date):
    """
    获取指定日期范围内的股票数据
    :param symbol: 股票符号
    :param start_date: 开始日期
    :param end_date: 结束日期
    :return: 股票数据 DataFrame，包含日期索引和价格列
    """
    # 第一步：创建日期范围
    dates = pd.date_range(start_date, end_date)
     # 第二步：初始化股票数据 DataFrame
    stock_data = get_data([symbol], dates)
    stock_data = stock_data.dropna()  # 删除任何缺失值的行
    stock_data = stock_data.sort_index()
    # 第八步：返回股票数据 DataFrame
    return stock_data

def initialize_trades(stock_data):
    """
    初始化交易 DataFrame
    :param stock_data: 股票数据 DataFrame
    :return: 交易操作 DataFrame
    """
    # 第一步：获取股票数据的日期索引
    dates = stock_data.index  # 提取股票数据的日期索引

    # 第二步：创建一个空的DataFrame，以股票数据的日期索引为索引
    trades = pd.DataFrame(index=dates)

    # 第三步：创建一个空的列表来存储交易操作
    trade_list = [0] * len(dates)  # 初始化交易操作列表，所有值为0

    # 第四步：将交易操作列表转换为Series
    trade_series = pd.Series(trade_list, index=dates)

    # 第五步：将交易操作Series添加到DataFrame中，列名为'Trade'
    trades['Trade'] = trade_series

    # 第六步：确保数据完整性
    trades = trades.dropna()  # 删除任何缺失值的行

    # 第七步：重设索引以确保数据的顺序
    trades = trades.sort_index()

    # 第八步：返回交易操作 DataFrame
    return trades
# 模块 3: 计算单日交易函数
def calculate_daily_trade(today_price, tomorrow_price, position):
    """
    计算单日交易
    :param today_price: 今天的价格
    :param tomorrow_price: 明天的价格
    :param position: 当前持仓
    :return: 当天的交易量，更新后的持仓
    """
    # 第一步：初始化交易量为0
    trade = 0  # Initialize the trade volume to 0

    # 第二步：检查明天的价格是否高于今天的价格
    if tomorrow_price > today_price:  # Check if tomorrow's price is higher than today's price
        # 第三步：如果当前持仓为0或持有空头仓位
        if position <= 0:  # If there is no position or a short position
            # 第四步：计算需要买入的股票数量
            # 假设我们每次最大持仓为1000股，所以我们需要买入1000股减去当前持仓量
            trade = 1000 - position  # Calculate the number of shares to buy to adjust the position to 1000 shares
            # 第五步：更新持仓为1000股
            position = 1000  # Update the position to 1000 shares

    # 第六步：检查明天的价格是否低于今天的价格
    elif tomorrow_price < today_price:  # Check if tomorrow's price is lower than today's price
        # 第七步：如果当前持仓为0或持有多头仓位
        if position >= 0:  # If there is no position or a long position
            # 第八步：计算需要卖出的股票数量
            # 假设我们每次最大持仓为-1000股（空头），所以我们需要卖出当前持仓量加上1000股
            trade = -1000 - position  # Calculate the number of shares to sell to adjust the position to -1000 shares
            # 第九步：更新持仓为-1000股
            position = -1000  # Update the position to -1000 shares

    # 第十步：返回当天的交易量和更新后的持仓
    return trade, position  # Return the trade volume and the updated position

# 模块 4: 计算理论上的最佳交易策略函数
def calculate_optimal_trades(stock_data):
    """
    计算理论上的最佳交易策略
    :param stock_data: 股票数据 DataFrame
    :return: 交易操作 DataFrame
    """
    # 第一步：初始化交易 DataFrame
    trades = initialize_trades(stock_data)  # Initialize the trades DataFrame

    # 第二步：初始化持仓为0
    position = 0  # Initialize the position to 0

    # 第三步：遍历每个交易日
    for current_date in stock_data.index[:-1]:  # Loop through each trading day except the last one
        # 第四步：获取今天和明天的价格
        today_price = stock_data.loc[current_date, 'price']  # Get today's price
        next_date = stock_data.index[stock_data.index.get_loc(current_date) + 1]  # Get the next trading day
        tomorrow_price = stock_data.loc[next_date, 'price']  # Get tomorrow's price

        # 第五步：计算单日交易
        trade, position = calculate_daily_trade(today_price, tomorrow_price, position)  # Calculate daily trade

        # 第六步：更新交易 DataFrame
        trades.loc[current_date, 'Trade'] = trade  # Update the trades DataFrame with the calculated trade

    # 第七步：返回交易操作 DataFrame
    return trades  # Return the trades DataFrame

# 模块 5: 主策略函数 testPolicy
def testPolicy(symbol, start_date, end_date, start_value):
    """
    实现理论上的最佳交易策略
    :param symbol: 股票符号
    :param start_date: 开始日期
    :param end_date: 结束日期
    :param start_value: 初始投资金额
    :return: 交易操作 DataFrame
    """
    # 获取股票数据
    stock_data = get_stock_data(symbol, start_date, end_date)

    # 计算最佳交易策略
    trades = calculate_optimal_trades(stock_data)

    # 返回交易操作 DataFrame
    return trades

# 模块 6: 主函数
def TheoreticallyOptimalStrategy():
    """
    主函数，用于测试理论上的最佳交易策略
    """
    symbol = 'JPM'  # 示例股票符号
    start_date = '2008-01-01'  # 示例开始日期
    end_date = '2009-12-31'  # 示例结束日期
    start_value = 100000  # 示例初始投资金额

    # 调用 testPolicy 函数获取交易操作 DataFrame
    trades = testPolicy(symbol, start_date, end_date, start_value)

    # 打印交易操作 DataFrame（可选）
    print(trades)

# 运行主函数
if __name__ == "__main__":
    TheoreticallyOptimalStrategy()
